# Narrative Analysis
In the Williamstown police department data, there are frequently "Narrative" fields associated with each incident. We have several overarching questions regarding these narratives, which include:
* Is there evidence of the "boys will be boys" culture instilled in the department by the previous police chief in the descriptions of incidents and interactions with people of varying demographic backgrounds and incident types? (e.g. do the narratives about noise complaints for parties differ from narratives about the police being called on a young woman?)
* Is there evidence that third shift is being more problematic (more tickets, etc) than other shifts? (This question is prompted by the recent revelation that the cop with the Hitler photo in his locker had been leading third shift cops in illegal searches of racial justice activists in the town)
* How much time in each shift is, on average, occupied by responding to calls? (This question is aimed at either supporting or disproving the hypothesis that there are too many cops in the department, and that potentially something like third shift could be handled by one on-call individual)
* How often and for what types of incidents does Williamstown PD respond to incidents at Williams College? (The PD listens to the Williams College dispatch and responds if the incidents are sever enough/may affect the town itself)

In this notebook I'll be relying on [spaCy](https://spacy.io/) to manipulate text. spaCy is a natural language processing (NLP) library that provides functionality for a variety of tasks.

## Sentiment analysis
In this section, we'll explore the use of [sentiment analysis](https://monkeylearn.com/sentiment-analysis/) to see if there is any difference in percieved sentiment across police narratives for different calls. Ideas for this section include:
* Associating incident type with narartive sentiment 
* Associating citizen and cop dempgraphic data with narrative sentiment 
* Associating specific officers with the average narrative sentiment of thier calls 

### Polarity analysis
Polarity analysis examines whether a text is positive, negative, or neutral in sentiment
#### spaCy TextBlob
[spaCy TextBlob](https://spacy.io/universe/project/spacy-textblob) is a spaCy pipeline component that connects spaCy's functionality to the [TextBlob](https://github.com/sloria/TextBlob) module.

In [3]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In this notebook I'll be manually pasting examples of narratives; but this will have to be scaled up once the OCR pipeline is done.

In [6]:
# Call 20-4
text = ("Activated Burglar Alarm. Key Holder Notified. Advised Dispatch that they have been having trouble with one of the zones. "
        "31 advised no need for key holder all is ok."
        "Checked building. Front and rear secure. All appears in order. Key holder was on phone with dispatch and satisfied.")
doc = nlp(text)

doc._.polarity, doc._.subjectivity

(0.11666666666666665, 0.8000000000000002)

From the [TextBlob docs](https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis): "The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective."

In [7]:
# Call 20-6
text = ("Active Structure fire. 1 party still inside at this time. "
        "Car 1 requesting to see if an officer can check the house aroud 0600 to make sure there are no signs of the fire reigniting."
        "House checked")
doc = nlp(text)

doc._.polarity, doc._.subjectivity

(0.18333333333333335, 0.7444444444444445)